In [1]:
from neo4j.v1 import GraphDatabase

driver = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j", "one"))

In [2]:
import pandas as pd

entities = pd.read_csv("../relation_extraction/basic/entities.csv").set_index("id")
types = pd.read_csv("../relation_extraction/basic/types.csv").set_index("id")
relationships = pd.read_csv("../relation_extraction/basic/relationships.csv", index_col = 0)
adv_relationships = pd.read_csv("../relation_extraction/basic/adv_relationships.csv", index_col = 0)

In [3]:
entities.head()

,name,page
id,,
1,1-up Doll,1-up_Doll.html
2,100th Ring,100th_Ring.html
3,15-second Game,15-second_Game.html
4,1986,1986.html
5,1987,1987.html


In [4]:
relationships.head()

,from,relationship,to
0,1,FIRST_APPEARANCE,8636
1,1,FIRST_APPEARANCE,6
2,1,FOUND,3766
3,1,USE,4550
4,2,FIRST_APPEARANCE,7747


In [5]:
adv_relationships.head()

,source,source_str,relation,name,dst_str,dst
0,1,1-up_Doll.html,item_from,Zelda II,NaN,NaN
1,1,1-up_Doll.html,live_in,which,NaN,NaN
2,2,100th_Ring.html,obtain_from,Vasu,Vasu.html,8270.0
3,2,100th_Ring.html,obtain_by,end,NaN,NaN
4,3,15-second_Game.html,locat_from,The Legend of Zelda,NaN,NaN


In [6]:
types.head()

,page,type
id,,
1,1-up_Doll.html,CHARACTER
1,1-up_Doll.html,ITEM
2,100th_Ring.html,ITEM
3,15-second_Game.html,LOCATION
36,2nd_Potion.html,ITEM


In [7]:
grouped_types = types.groupby(types.index)["type"].apply(set).to_frame()

In [8]:
import numpy as np
joint = pd.merge(entities, grouped_types, left_index=True, right_index=True, how='outer')
joint.tail()

,name,page,type
id,,,
8709,Zunari,Zunari.html,{CHARACTER}
8710,Zuta,Zuta.html,"{LOCATION, CHARACTER}"
8711,Zyle,Zyle.html,{CHARACTER}
8712,?,_.html,NaN
8713,???,___.html,"{LOCATION, CHARACTER, ITEM}"


In [9]:
joint.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8713 entries, 1 to 8713
Data columns (total 3 columns):
name    8713 non-null object
page    8713 non-null object
type    6348 non-null object
dtypes: object(3)
memory usage: 272.3+ KB


In [10]:
import json

def escape(string):
    return json.dumps(string)[1:-1]
print(joint.loc[8713]['name'])
escape(joint.loc[8713]['name'])

":".join(joint.loc[8713]["type"])

???


'LOCATION:CHARACTER:ITEM'

In [12]:
create_template = "CREATE (a:%s {id:%d, name:\"%s\", page:\"%s\"})"
with driver.session() as session:
    for i, row in joint.iterrows():
        type_ = 'OBJECT' if pd.isnull(row['type']) else "OBJECT:"+":".join(row['type'])
        id_ = row.name
        name = escape(row['name'])
        page = row['page']
        insert_stmt = create_template % (type_, id_, name, page) 
        
        result = session.run(insert_stmt)
    session.run("CREATE INDEX ON :ENTITY(id)")

In [15]:
relationship_template = """MATCH (from:OBJECT{id:%d}),(to:OBJECT{id:%d})
CREATE (from)-[:%s]->(to)"""

with driver.session() as session:
    for i, row in relationships.loc[:32299].iterrows():
        if i % 500 == 0:
            print(i)
        try:
            relationship_stmt = relationship_template % (row["from"], row["to"], row["relationship"])
            session.run(relationship_stmt)
        except Exception as inst:
            print("Error", i)
            print(inst)
            break

0
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500
8000
8500
9000
9500
10000
10500
11000
11500
12000
12500
13000
13500
14000
14500
15000
15500
16000
16500
17000
17500
18000
18500
19000
19500
20000
20500
21000
21500
22000
22500
23000
23500
24000
24500
25000
25500
26000
26500
27000
27500
28000
28500
29000
29500
30000
30500
31000
31500
32000
